"Требования к нереляционным БД

2 - нереляционная БД

2 - красивая структура БД

2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)

2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)

2 - Neo4j - нахождение путей не только к соседним вершинам, операции с графами"


In [1]:
from pprint import pprint
from neo4j import GraphDatabase, basic_auth

In [2]:
driver = GraphDatabase.driver(
    "neo4j+s://2bb967b7.databases.neo4j.io:7687", 
    auth=basic_auth("neo4j", "R4t4Mta4Z-qidT1aY6UNWUH701RJD1oAFfubc-gWJG8"))
session = driver.session()

## 1.1 Create, Read

In [8]:
name = str(input('Введите ваше имя: '))
age = str(input('Сколько вам лет? '))
city, country = str(input('Где вы живете? В формате город, страна: ')).split(', ')
interests = str(input('Какие книги вам интересны? ')).split(', ')

Введите ваше имя: Юлия Пыжак
Сколько вам лет? 22
Где вы живете? В формате город, страна: Москва, Россия
Какие книги вам интересны? Политика, Детективы


In [10]:
def add_new_person(name, age, city, country, interests):
    find_city = """MERGE (c:Country {name:'%s'})""" % country # merge - match if exists, create if doesn't
    session.run(find_city)
    add_person = """CREATE (p:Person {name:'%s', age:'%s'})""" % (name, age)
    session.run(add_person)
    
    add_edge = """MATCH (a:Person {name:'%s'}),
                (c:Country {name:'%s'})
                CREATE (a)-[:LIVES_IN {city:'%s'}]->(c)""" % (name, country, city)
    session.run(add_edge)
    
    for interest in interests:
        add_interest = """MERGE (i:Interest {name:'%s'})""" % interest
        session.run(add_interest)
        edge_interest = """MATCH (p:Person {name:'%s'}),
                            (i:Interest {name:'%s'}) 
                            CREATE (p)-[:INTERESTED_IN]->(i)""" % (name, interest)
        session.run(edge_interest)
        
    get_inserted_data = """MATCH (p:Person {name:'%s'})-[r:LIVES_IN]-(a) RETURN p.name, p.age, r.city, a.name""" % name
    r = session.run(get_inserted_data)
    result1 = [record.data() for record in r]
    result1 = result1[0]
    
    get_inserted_data = """MATCH (p:Person {name:'%s'})-[r:INTERESTED_IN]-(a) RETURN a.name""" % name
    r = session.run(get_inserted_data)
    result2 = [record.data() for record in r]
    interests1 = [interest['a.name'].lower() for interest in result2]
    
    print('Запись сделана. \nИмя: {} \nВозраст: {} \nГород: {}\nСтрана: {}\nИнтересы: {}'.format(result1['p.name'],
                                                                               result1['p.age'],
                                                                               result1['r.city'],
                                                                               result1['a.name'],
                                                                               ', '.join(interests1)))

In [11]:
add_new_person(name, age, city, country, interests)

Запись сделана. 
Имя: Юлия Пыжак 
Возраст: 22 
Город: Москва
Страна: Россия
Интересы: детективы, политика


## 1.2 Update 

поменяем город

In [12]:
city = 'Рязань'

In [13]:
update_city = """MATCH (:Person {name: '%s'})-[r:LIVES_IN]-(:Country {name: '%s'})
                    SET r.city = '%s'
                    RETURN r.city""" % (name, country, city)

result = session.run(update_city)

In [14]:
for res in result:
    print(res.data()['r.city'])

Рязань


# 1.3 Delete

In [15]:
interest_to_del = 'Политика'

In [16]:
delete_interest = """MATCH (:Person {name: '%s'})-[r:INTERESTED_IN]->(:Interest {name: '%s'})
                    DELETE r""" % (name, interest_to_del)
session.run(delete_interest)

In [17]:
q = """MATCH (:Person {name: '%s'})-[r:INTERESTED_IN]->(i) 
        RETURN i.name""" % name
new_interests = session.run(q)

In [18]:
for i in new_interests:
    print(i.data()['i.name'])

Детективы


## 2. Аггрегации

Cортировка - кто отправил/получил больше всех книжек, когда отправлена первая/последняя книжка

аггрегация - среднее количесвто отправленных книжек, средний возраст

группировка - среднее количесвто населения по странам, самый поппулярный интерес по странам

In [19]:
# пользователь, который отправил больше всех книжек
q = """MATCH (p:Person)-[r:SENT_BOOK_TO]->(:Person) 
        RETURN p.name, count(r) AS cnt
        ORDER BY cnt DESC
        LIMIT 1"""
result = session.run(q)

In [20]:
for res in result:
    print(res.data()['p.name'], res.data()['cnt'], sep=', ')

Моисеева Елена, 4


In [3]:
q = """MATCH (p:Person)
        RETURN avg(tointeger(p.age)) as avg_age"""
result = session.run(q)

In [5]:
print(result.data())

[{'avg_age': 50.196078431372534}]


In [18]:
# когда отправлена первая книжка
q = """MATCH (p1:Person)-[r:SENT_BOOK_TO]->(p2:Person),
        (p1)-[t1:LIVES_IN]->(c1:Country),
        (p2)-[t2:LIVES_IN]->(c2:Country)
        RETURN p1.name AS from, p2.name AS to, date(r.date) AS date,
                r.book_name AS book, t1.city AS from_city,
                c1.name as from_country, t2.city AS to_city,
                c2.name as to_country
        ORDER BY r.date
        LIMIT 1"""
result = session.run(q)

In [20]:
for res in result:
    pprint(res.data())

{'book': 'Ночь в Лиссабоне',
 'date': neo4j.time.Date(2021, 12, 11),
 'from_city': 'Вердан',
 'from_country': 'Канада',
 'p1.name': 'Смирнов Даниил',
 'p2.name': 'Воробьева Арина',
 'to': 'Темпл',
 'to_country': 'США'}


In [196]:
# самый популярный интерес в России
popular_interest = """MATCH (i:Interest)<-[r:INTERESTED_IN]-(p:Person)-[r1:LIVES_IN]->(c) 
                        WITH c.name as city, i.name as interest, count(i.name) as cnt
                        WHERE c.name='Россия' 
                        RETURN interest, cnt 
                        ORDER BY cnt DESC"""
result = session.run(popular_interest)

In [197]:
for res in result:
    print(res.data()['interest'], res.data()['cnt'], sep=', ')

Зарубежная литература, 2
Детективы, 1
Романы, 1


In [202]:
# какиех авторов чаще всего отправляют
popular_authors_sent = """MATCH (:Person)-[r:SENT_BOOK_TO]-(:Person)
                        RETURN r.author as author, collect(DISTINCT r.genre) AS genre, count(r.author) as cnt
                        ORDER BY cnt DESC
                        LIMIT 5"""
result = session.run(popular_authors_sent)

In [203]:
for res in result:
    print(res.data()['author'], res.data()['genre'], res.data()['cnt'], sep=', ')

Марисса Мейер, ['Киберпанк', 'Зарубежное фэнтези', 'Космическая фантастика'], 10
Керри Манискалко, ['Зарубежные детективы'], 10
Эрих Ремарк, ['Классическая зарубежная литература'], 8
Холли Блэк, ['Фантастика и фэнтези для подростков', 'Зарубежное фэнтези'], 6
Донна Тартт, ['Современная зарубежная проза'], 6


In [23]:
q = """MATCH (p1:Person)-[:SENT_BOOK_TO]->(p2:Person),
        (p1)-[:LIVES_IN]->(c1),
        (p2)-[:LIVES_IN]->(c2)
        WHERE c1 = c2
        RETURN p1.name AS from, p2.name AS to, c1.name AS country"""
result = session.run(q)

In [24]:
for res in result:
    pprint(res.data())

{'country': 'Индия', 'from': 'Платонова Есения', 'to': 'Чистякова Валерия'}
{'country': 'Южная Корея', 'from': 'Степанова Нина', 'to': 'Коновалова Кира'}
{'country': 'Индия', 'from': 'Чистякова Валерия', 'to': 'Матвеев Вадим'}
{'country': 'Испания', 'from': 'Еремина Милана', 'to': 'Цветков Егор'}
{'country': 'Грузия', 'from': 'Кравцова Анастасия', 'to': 'Нестеров Павел'}
{'country': 'Канада', 'from': 'Смирнов Даниил', 'to': 'Казакова Алиса'}
{'country': 'Канада', 'from': 'Моисеева Елена', 'to': 'Казакова Алиса'}
{'country': 'Камерун', 'from': 'Щеглова Полина', 'to': 'Островская Николь'}
{'country': 'Испания', 'from': 'Лебедев Максим', 'to': 'Тимофеева Валерия'}


In [7]:
q = """MATCH (p:Person)
        RETURN round(avg(apoc.node.degree.out(p, 'SENT_BOOK_TO')), 2) as avg_n_books"""
result = session.run(q)
print(result.data())

[{'avg_n_books': 1.33}]


In [3]:
q = """MATCH p=ShortestPath((p1:Person {name:'Дубровин Ярослав'})
        -[:SENT_BOOK_TO|LIVES_IN*1..50]->(c1:Country {name:'Россия'}))
        RETURN p"""
result = session.run(q)

In [4]:
for res in result:
    pprint(res.data())

{'p': [{'age': '79', 'name': 'Дубровин Ярослав'},
       'SENT_BOOK_TO',
       {'age': '22', 'name': 'Федотова Елизавета'},
       'SENT_BOOK_TO',
       {'age': '49', 'name': 'Горюнова Варвара'},
       'LIVES_IN',
       {'name': 'Россия'}]}
